In [5]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets

# Sample product database
products = {
    'ID': [1, 2, 3, 4, 5],
    'Name': ['Laptop', 'Smartphone', 'Headphones', 'Smart Watch', 'Tablet'],
    'Price': [999.99, 699.99, 149.99, 199.99, 349.99],
    'Stock': [10, 15, 20, 12, 8]
}

product_df = pd.DataFrame(products)

# Shopping cart
cart_df = pd.DataFrame({'Items': [], 'Quantity': [], 'Price': []})

# UI Components
product_dropdown = widgets.Dropdown(
    options=[(f"{row['Name']} (${row['Price']})", row['ID']) for _, row in product_df.iterrows()],
    description='Product:'
)

quantity_input = widgets.BoundedIntText(
    value=1,
    min=1,
    max=10,
    description='Quantity:'
)

add_to_cart_btn = widgets.Button(description="Add to Cart", button_style='success')
view_cart_btn = widgets.Button(description="View Cart", button_style='info')
checkout_btn = widgets.Button(description="Proceed to Checkout", button_style='warning')
back_btn = widgets.Button(description="Back to Products", button_style='primary')

output = widgets.Output()

# Display functions
def display_products():
    with output:
        clear_output()
        print("🛍 Welcome to Our E-Commerce Store!")
        print("\n📦 Available Products:")
        styled_df = product_df[['Name', 'Price', 'Stock']].style.set_properties(
            subset=['Name', 'Price', 'Stock'],
            **{
                'background-color': '#f8f9fa',
                'color': '#212529',
                'border-color': '#dee2e6'
            }
        )
        display(styled_df)
        
        display(widgets.VBox([
            product_dropdown,
            quantity_input,
            widgets.HBox([add_to_cart_btn, view_cart_btn])
        ]))

def display_cart():
    with output:
        clear_output()
        if cart_df.empty:
            print("🛒 Your cart is empty")
        else:
            print("🛒 Your Shopping Cart:")
            cart_df['Total'] = cart_df['Quantity'] * cart_df['Price']
            display(cart_df.style.set_properties(
                subset=['Items', 'Quantity', 'Price', 'Total'],
                **{
                    'background-color': '#f8f9fa',
                    'color': '#212529',
                    'border-color': '#dee2e6'
                }
            ))
            
            total = (cart_df['Quantity'] * cart_df['Price']).sum()
            print(f"\n💵 Total: ${total:.2f}")
            
        display(widgets.VBox([
            checkout_btn,
            back_btn
        ]))

def display_checkout():
    with output:
        clear_output()
        if cart_df.empty:
            print("Your cart is empty. Nothing to checkout.")
            display(back_btn)
        else:
            total = (cart_df['Quantity'] * cart_df['Price']).sum()
            print("💰 Checkout Summary")
            print("===================")
            display(cart_df)
            print(f"\nTotal Amount: ${total:.2f}")
            
            print("\nPlease enter your shipping information:")
            name = widgets.Text(description="Name:", style={'description_width': '100px'})
            address = widgets.Textarea(description="Address:", style={'description_width': '100px'})
            email = widgets.Text(description="Email:", style={'description_width': '100px'})
            payment = widgets.Dropdown(
                options=['Credit Card', 'PayPal', 'Bank Transfer'],
                description='Payment:',
                style={'description_width': '100px'}
            )
            
            confirm_btn = widgets.Button(description="Confirm Order", button_style='success')
            
            form = widgets.VBox([
                name,
                address,
                email,
                payment,
                confirm_btn,
                back_btn
            ])
            
            def on_confirm_click(b):
                with output:
                    clear_output()
                    if not name.value or not address.value or not email.value:
                        print("⚠ Please fill in all fields!")
                        display(form)
                        return
                    
                    print("🎉 Order Placed Successfully!")
                    print(f"\nThank you, {name.value}!")
                    print("\nYour order will be shipped to:")
                    print(address.value)
                    print(f"\nA confirmation has been sent to {email.value}")
                    print("\nPayment method:", payment.value)
                    print("\nWe appreciate your business!")
                    
                    # Clear the cart
                    global cart_df
                    cart_df = pd.DataFrame({'Items': [], 'Quantity': [], 'Price': []})
                    
                    display(back_btn)
            
            confirm_btn.on_click(on_confirm_click)
            display(form)

# Event handlers
def on_add_to_cart_click(b):
    product_id = product_dropdown.value
    quantity = quantity_input.value
    
    product = product_df[product_df['ID'] == product_id].iloc[0]
    
    if quantity > product['Stock']:
        with output:
            print(f"⚠ Only {product['Stock']} available in stock!")
        return
    
    # Add to cart
    global cart_df
    if product['Name'] in cart_df['Items'].values:
        # Update quantity if product already in cart
        cart_df.loc[cart_df['Items'] == product['Name'], 'Quantity'] += quantity
    else:
        # Add new item to cart
        new_item = pd.DataFrame({
            'Items': [product['Name']],
            'Quantity': [quantity],
            'Price': [product['Price']]
        })
        cart_df = pd.concat([cart_df, new_item], ignore_index=True)
    
    # Update stock
    product_df.loc[product_df['ID'] == product_id, 'Stock'] -= quantity
    
    with output:
        print(f"✅ Added {quantity} {product['Name']}(s) to cart")

def on_view_cart_click(b):
    display_cart()

def on_checkout_click(b):
    display_checkout()

def on_back_click(b):
    display_products()

# Assign event handlers
add_to_cart_btn.on_click(on_add_to_cart_click)
view_cart_btn.on_click(on_view_cart_click)
checkout_btn.on_click(on_checkout_click)
back_btn.on_click(on_back_click)

# Initialize display
display_products()
display(output)


Output()